# Breaking round-reduced Trivium using algebraic cryptanalysis

In [6]:
from __future__ import print_function
from collections import Counter

__S0_OFFSET__ = 0
__S1_OFFSET__ = 93
__S2_OFFSET__ = 93 + 84

def flip_constant(constant):
    constant = (constant + 1) % 2
    return constant

def current_S0(i_key, j_bit):
    if(j_bit >= i_key):
        return [""+str(j_bit - i_key + __S0_OFFSET__ + 1)]
    else:
        return FS0(i_key - j_bit - 1)
def current_S1(i_key, j_bit):
    if(j_bit >= i_key):
        return [""+str(j_bit - i_key + __S1_OFFSET__ + 1)]
    else:
        return FS1(i_key - j_bit - 1)
    
def current_S2(i_key, j_bit):
    if(j_bit >= i_key):
        return [""+str(j_bit - i_key + __S2_OFFSET__ + 1)]
    else:
        return FS2(i_key - j_bit - 1)

def FS0(i_key):
    resList = (current_S2(i_key, (243 - __S2_OFFSET__ - 1))) 
    term1 = (current_S2(i_key, (286 - __S2_OFFSET__ - 1)))
    term2 = (current_S2(i_key, (287 - __S2_OFFSET__ - 1)))
    for i_term1 in term1:
        for i_term2 in term2:
            resList.append(i_term1+"*"+i_term2)
    
    resList = resList + (current_S2(i_key, (288 - __S2_OFFSET__ - 1))) + (current_S0(i_key, (69 - __S0_OFFSET__ - 1)))
    return resList  

def FS1(i_key):
    resList = (current_S0(i_key, (66 - __S0_OFFSET__ - 1))) 
    term1 = (current_S0(i_key, (91 - __S0_OFFSET__ - 1)))
    term2 = (current_S0(i_key, (92 - __S0_OFFSET__ - 1)))
    for i_term1 in term1:
        for i_term2 in term2:
            resList.append(i_term1+"*"+i_term2)
    
    resList = resList + (current_S0(i_key, (93 - __S0_OFFSET__ - 1))) + (current_S1(i_key, (171 - __S1_OFFSET__ - 1)))
    return resList  

def FS2(i_key):
    resList = (current_S1(i_key, (162 - __S1_OFFSET__ - 1))) 
    term1 = (current_S1(i_key, (175 - __S1_OFFSET__ - 1)))
    term2 = (current_S1(i_key, (176 - __S1_OFFSET__ - 1)))
    for i_term1 in term1:
        for i_term2 in term2:
            resList.append(i_term1+"*"+i_term2)
    
    resList = resList + (current_S1(i_key, (177 - __S1_OFFSET__ - 1))) + (current_S2(i_key, (264 - __S2_OFFSET__ - 1)))
    return resList  
    
def keystream(i_key):
    return (current_S0(i_key, (66 - __S0_OFFSET__ - 1))) + (current_S0(i_key, (93 - __S0_OFFSET__ - 1))) + (current_S1(i_key, (162 - __S1_OFFSET__ - 1))) + (current_S1(i_key, (177 - __S1_OFFSET__ - 1))) + (current_S2(i_key, (243 - __S2_OFFSET__ - 1))) + (current_S2(i_key, (288 - __S2_OFFSET__ - 1)))

nb_rounds = 80
keystream_str = "11000110100011011001000100101100010001010101011000011100001010100111001110111101001100000110110000111000100001111111111000001001011101110110110000010110110000001100011100101100000110010000000011111111"
initial_state = "000111111000101000100001100010111101111110111001001101110001111110111011011010100000000000000010100100111000101011110100000010101101101110001000010100001101010001111001010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111"

for i in range(0,40):
    constant = 1
    if(keystream_str[i]=="0"):
        constant = flip_constant(constant)
    clause_str = keystream(i+nb_rounds)
    #print(clause_str)
    monomials = []
    for mon in clause_str:
        all_terms = set(mon.split('*'))
        terms = []
        zero = 0
        for term in all_terms:
            if(int(term) > 80):
                if(initial_state[int(term)-1] == "0"):
                    zero = 1
                    break
            else:
                terms.append(term)
        if(zero==0):
            if(len(terms) > 0):
                terms.sort()
                monomials.append(terms)
            else:
                constant = flip_constant(constant)
    monomials.sort()
    #monomials_set = [monomials[i] for i in range(len(monomials)) if i == 0 or monomials[i] != monomials[i-1]]
    #delete duplicates
    monomials_set = []
    i = 0
    while (i < len(monomials)):
        if(i == len(monomials) - 1):
            monomials_set.append(monomials[i])
        else:
            if(monomials[i]!=monomials[i+1]):
                monomials_set.append(monomials[i])
            else:
                i=i+1
        i=i+1
    print("x", end = '')
    if(constant==1):
        print(" T", end = '')
    for mon in monomials_set:
        if(len(mon) > 1):
            print(" ." + str(len(mon)), end = '')
        for term in mon:
            print(" "+str(term), end = '')
    print(" 0")



x 13 0
x 12 .2 79 80 0
x T 11 .2 78 79 80 0
x 10 .2 77 78 79 0
x T 66 .2 76 77 78 9 0
x T 65 .2 75 76 77 8 0
x 64 7 .2 74 75 76 0
x 6 63 .2 73 74 75 0
x 5 62 .2 72 73 74 0
x T 4 61 .2 71 72 73 0
x 3 60 .2 70 71 72 0
x 2 59 .2 69 70 71 0
x 1 58 .2 68 69 70 0
x T 57 .2 67 68 0
x 56 .2 66 67 0
x 55 .2 65 66 0
x 54 .2 64 65 .2 79 80 0
x 53 .2 63 64 .2 78 79 80 0
x T 52 .2 62 63 .2 77 78 79 0
x 51 .2 61 62 .2 76 77 78 0
x 50 .2 60 61 .2 75 76 77 0
x 49 .2 59 60 .2 74 75 76 0
x 48 .2 58 59 .2 73 74 75 0
x T 47 .2 57 58 .2 72 73 74 0
x T 46 .2 56 57 .2 71 72 73 0
x 45 .2 55 56 .2 70 71 72 0
x T 44 .2 54 55 .2 69 70 71 0
x T 43 .2 53 54 .2 68 69 70 0
x T 42 .2 52 53 .2 67 68 69 0
x T 41 .2 51 52 .2 66 67 68 0
x T 40 .2 50 51 .2 65 66 67 0
x T 39 .2 49 50 .2 64 65 66 0
x 38 .2 48 49 .2 63 64 65 0
x 37 .2 47 48 .2 62 63 64 0
x T 36 .2 46 47 .2 61 62 63 0
x T 35 .2 45 46 .2 60 61 62 0
x T 34 .2 44 45 .2 59 60 61 0
x T 33 .2 43 44 .2 58 59 60 0
x T 32 .2 42 43 .2 57 58 59 0
x 31 .2 41 42 .2 56 57 

## Your solution

In [ ]:
# Functions used for the attack
# Starting only from an MQ-Sign public key, try to find the value of S in the upper-right corner.
# As per the equivalent keys optimization, this is a v by m matrix. 
def GetSolution(P):
    
    return S_sol


## Test time

In [ ]:
F = MQSignCentralMap()
S = UpperTriangularS()
P = GetPublicKey(F, S)

S_sol = GetSolution(P)
print(S_sol == S.submatrix(0, v, v, m))